<a href="https://colab.research.google.com/github/curlos/manga-panel-splitter/blob/main/Magi_V2_Model_Analyze_Images_(Manga_Panel_Splitter).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.40 pulp pyngrok python-dotenv accelerate psutil

In [10]:
!pkill -f ngrok

In [ ]:
import requests
from flask import Flask, request, jsonify
import os
import numpy as np
from transformers import AutoModel
import torch
from pyngrok import ngrok
import threading
import base64
import pdb
from google.colab import drive
from dotenv import load_dotenv
import socket
import subprocess
import psutil
import logging
import sys
import signal

# Mount Google Drive if it hasn't mounted already.
if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

# Path to the .env file in your Drive
env_path = '/content/drive/MyDrive/google_colab_env_files/.env'

# Load the .env file
load_dotenv(env_path)

# Access the ngrok auth token
ngrok_auth_token = os.getenv("NGROK_AUTH_TOKEN")

if not ngrok_auth_token:
    raise ValueError("NGROK_AUTH_TOKEN is not set. Check your .env file in Google Drive ('google_colab_env_files/.env').")

ngrok.set_auth_token(ngrok_auth_token)

port = 8888

# Configure Flask logging to use sys.stdout
logging.basicConfig(
    stream=sys.stdout,
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s]: %(message)s",
)
logger = logging.getLogger()

app = Flask(__name__)
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # Increase limit to 16MB

# Initialize the Magi model
magi_model = AutoModel.from_pretrained(
            "ragavsachdeva/magiv2", trust_remote_code=True).eval()

def get_per_page_results(magi_model, chapter_pages, character_bank):
  # Set to "no_grad()" so that there's inference without tracking gradients. Basically, this saves memory and computational resources by turning off gradient tracking.
  with torch.no_grad():
      per_page_results = magi_model.do_chapter_wide_prediction(
          chapter_pages, character_bank, use_tqdm=True, do_ocr=True
      )

  return per_page_results

def is_port_in_use(port):
  with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
      return s.connect_ex(("0.0.0.0", port)) == 0

def kill_process_on_port(port):
  try:
      # Find the process ID (PID) using netstat and taskkill
      result = subprocess.check_output(
          f"netstat -ano | findstr :{port}", shell=True
      ).decode()
      pid = int(result.strip().split()[-1])

      # Kill the process
      subprocess.call(["taskkill", "/F", "/PID", str(pid)])
      print(f"Port {port} is now free (process {pid} killed).")
  except subprocess.CalledProcessError:
      print(f"No process is using port {port}.")
  except Exception as e:
      print(f"Error freeing port {port}: {e}")

@app.route('/process-images-with-magi-model', methods=['POST'])
def process_images_with_magi_model():
  logger.info("Received request to /process-images-with-magi-model")

  try:
    # Parse JSON payload
    request_data = request.json
    encoded_arrays = request_data.get("chapter_pages_image_numpy_array")
    character_bank = request_data.get("character_bank")

    # Decode and reconstruct the arrays
    chapter_pages_image_numpy_array = [
        np.frombuffer(base64.b64decode(item["data"]), dtype=item["dtype"]).reshape(item["shape"])
        for item in encoded_arrays
    ]

    # Run Magi model on the file
    per_page_results = get_per_page_results(
        magi_model, chapter_pages_image_numpy_array, character_bank
    )

    return per_page_results
  except Exception as e:
    return jsonify({"error": str(e)}), 500

@app.route('/hello-world', methods=['GET'])
def hello_world():
  logger.info("Hello World endpoint hit")
  return 'Hello World!'

def run_flask():
  logger.info("Starting Flask server...")
  # Run Flask app without reloader (important for threading)
  app.run(host="0.0.0.0", port=port, debug=True, use_reloader=False)


def force_free_port(port):
    """
    Forcefully free a port by identifying and killing the process using it,
    while avoiding critical processes like Colab kernel processes.
    """
    for conn in psutil.net_connections(kind="inet"):
        if conn.laddr.port == port:
            try:
                # Get the process using the port
                process = psutil.Process(conn.pid)
                process_name = process.name().lower()
                process_cmdline = " ".join(process.cmdline())

                # Skip Colab kernel-related processes
                if "colab" in process_name or "python" in process_name and "kernel" in process_cmdline:
                    print(f"Skipping Colab or kernel-related process: {process_name} (PID {conn.pid})")
                    continue

                # Kill the process
                print(f"Killing process using port {port}: {process_name} (PID {conn.pid})")
                os.kill(conn.pid, signal.SIGKILL)
                print(f"Successfully killed process {process_name} (PID {conn.pid}).")
            except psutil.NoSuchProcess:
                print(f"No such process exists for PID {conn.pid}.")
            except psutil.AccessDenied:
                print(f"Permission denied to kill process using port {port}: PID {conn.pid}")
            except Exception as e:
                print(f"Failed to kill process on port {port}: {e}")
            return
    print(f"Port {port} is not in use.")

def stop_all_tunnels():
  try:
    # Fetch all active tunnels
    response = requests.get(f"http://127.0.0.1:{port}/api/tunnels")
    tunnels = response.json().get("tunnels", [])

    # Terminate each tunnel
    for tunnel in tunnels:
        tunnel_name = tunnel["name"]
        delete_url = f"http://127.0.0.1:{port}/api/tunnels/{tunnel_name}"
        requests.delete(delete_url)
        print(f"Terminated tunnel: {tunnel_name}")
  except Exception as e:
    print(f"Failed to terminate tunnels: {e}")


if __name__ == '__main__':
  # Check and free the port if in use
  force_free_port(port)

  # Start Flask in a thread to prevent blocking
  thread = threading.Thread(target=run_flask)
  thread.start()

  # Expose the Flask app to the internet using ngrok
  public_url = ngrok.connect(port)
  print("Public URL:", public_url)

  # Keep the Colab cell running
  while True:
      pass
